<a href="https://colab.research.google.com/github/Y4-Deep-Learning-Assignment/Pneumonia-_Detection/blob/Xception/Xception_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xception Transfer Learning for Pneumonia Detection

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Install and import required libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow keras numpy pandas matplotlib seaborn scikit-learn -q

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

import os
import zipfile
from google.colab import files
import time
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

# Check GPU availability
print("\nGPU Available:", tf.test.is_gpu_available())
if tf.test.is_gpu_available():
    print("GPU Device:", tf.test.gpu_device_name())

## Extract Dataset

In [ ]:
import zipfile
import shutil

zip_path = "/content/drive/MyDrive/DL Assignment/archive.zip"

shutil.copy(zip_path, "/content/")

# Extract the dataset
print("Extracting dataset...")
with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall('/content')

print("Dataset structure:")
!find /content/chest_xray -type d -print


## Explore the Dataset

In [ ]:
dataset_path = '/content/chest_xray'
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')
val_path = os.path.join(dataset_path, 'val')

def explore_dataset():
    print("Dataset Exploration:")
    print("=" * 50)

    for split in ['train', 'test', 'val']:
        split_path = os.path.join(dataset_path, split)
        normal_path = os.path.join(split_path, 'NORMAL')
        pneumonia_path = os.path.join(split_path, 'PNEUMONIA')

        normal_count = len(os.listdir(normal_path))
        pneumonia_count = len(os.listdir(pneumonia_path))
        total_count = normal_count + pneumonia_count

        print(f"\n{split.upper()} SET:")
        print(f"  Normal images: {normal_count}")
        print(f"  Pneumonia images: {pneumonia_count}")
        print(f"  Total images: {total_count}")
        print(f"  Pneumonia ratio: {pneumonia_count/total_count:.2%}")

explore_dataset()

In [ ]:
# Visualize sample images
def visualize_samples():
    """Display sample images from both classes"""
    fig, axes = plt.subplots(2, 4, figsize=(15, 8))

    # Normal samples
    normal_path = os.path.join(train_path, 'NORMAL')
    normal_images = [f for f in os.listdir(normal_path) if f.endswith(('.jpeg', '.jpg', '.png'))][:4]

    for i, img_name in enumerate(normal_images):
        img_path = os.path.join(normal_path, img_name)
        img = plt.imread(img_path)
        axes[0, i].imshow(img, cmap='gray')
        axes[0, i].set_title(f'Normal\n{img_name}', fontsize=10)
        axes[0, i].axis('off')

    # Pneumonia samples
    pneumonia_path = os.path.join(train_path, 'PNEUMONIA')
    pneumonia_images = [f for f in os.listdir(pneumonia_path) if f.endswith(('.jpeg', '.jpg', '.png'))][:4]

    for i, img_name in enumerate(pneumonia_images):
        img_path = os.path.join(pneumonia_path, img_name)
        img = plt.imread(img_path)
        axes[1, i].imshow(img, cmap='gray')
        axes[1, i].set_title(f'Pneumonia\n{img_name}', fontsize=10)
        axes[1, i].axis('off')

    plt.suptitle('Sample Chest X-Ray Images - Xception Model', fontsize=16, y=0.95)
    plt.tight_layout()
    plt.show()

visualize_samples()

## Data Preprocessing and Augmentation

In [ ]:
IMG_HEIGHT = 299  # Xception expects 299x299 images
IMG_WIDTH = 299
BATCH_SIZE = 32

print("Setting up data generators for Xception...")

# Enhanced data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Increased from 15
    width_shift_range=0.15,  # Increased from 0.1
    height_shift_range=0.15,  # Increased from 0.1
    horizontal_flip=True,
    zoom_range=0.25,  # Increased from 0.2
    brightness_range=[0.8, 1.2],  # Wider range
    shear_range=0.15,  # Added shear
    fill_mode='constant',
    cval=0
)

# Only rescaling for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Create proper validation set by splitting training data
import tempfile
import shutil
from sklearn.model_selection import train_test_split

def create_proper_validation_split(validation_size=0.15):
    """
    Split training data to create a proper validation set
    """
    print("Creating proper validation split...")

    # Create temporary directories
    base_temp_dir = tempfile.mkdtemp()
    new_train_dir = os.path.join(base_temp_dir, 'train')
    new_val_dir = os.path.join(base_temp_dir, 'val')

    # Create subdirectories
    for split_dir in [new_train_dir, new_val_dir]:
        os.makedirs(os.path.join(split_dir, 'NORMAL'), exist_ok=True)
        os.makedirs(os.path.join(split_dir, 'PNEUMONIA'), exist_ok=True)

    # Split each class
    for class_name in ['NORMAL', 'PNEUMONIA']:
        source_dir = os.path.join(train_path, class_name)
        images = [f for f in os.listdir(source_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]

        # Split images
        train_imgs, val_imgs = train_test_split(
            images,
            test_size=validation_size,
            random_state=42,
            shuffle=True
        )

        # Copy images to new directories
        for img in train_imgs:
            shutil.copy2(
                os.path.join(source_dir, img),
                os.path.join(new_train_dir, class_name, img)
            )
        for img in val_imgs:
            shutil.copy2(
                os.path.join(source_dir, img),
                os.path.join(new_val_dir, class_name, img)
            )

        print(f"  {class_name}: {len(train_imgs)} train, {len(val_imgs)} validation")

    return new_train_dir, new_val_dir, base_temp_dir

# Create proper train/validation split
new_train_dir, new_val_dir, temp_dir = create_proper_validation_split(validation_size=0.15)

# Create IMPROVED data generators with proper validation set
train_generator = train_datagen.flow_from_directory(
    new_train_dir,  # Use new training directory
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    shuffle=True,
    seed=42
)

validation_generator = val_test_datagen.flow_from_directory(
    new_val_dir,  # Use new validation directory
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

# Keep original test set unchanged
test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    color_mode='rgb',
    shuffle=False
)

# Display class information
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

print(f"\nIMPROVED Data Generators Created Successfully!")
print(f"Class indices: {class_indices}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")  # Now proper size!
print(f"Test samples: {test_generator.samples}")
print(f"Class names: {class_names}")

# Calculate class weights based on NEW training data distribution
normal_count = len(os.listdir(os.path.join(new_train_dir, 'NORMAL')))
pneumonia_count = len(os.listdir(os.path.join(new_train_dir, 'PNEUMONIA')))
total = normal_count + pneumonia_count

weight_for_0 = total / (2 * normal_count)    # Weight for NORMAL
weight_for_1 = total / (2 * pneumonia_count)  # Weight for PNEUMONIA

class_weights = {0: weight_for_0, 1: weight_for_1}
print(f"Class weights: {class_weights}")


## Build Xception Transfer Learning Model

In [ ]:
def create_xception_model():
    """
    Create Xception transfer learning model for pneumonia detection
    Xception is known for its depthwise separable convolutions
    """
    print("Building Xception model...")

    # Load pre-trained Xception
    base_model = Xception(
        weights='imagenet',       # Pre-trained on ImageNet
        include_top=False,        # Exclude the final classification layers
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # Freeze the base model layers initially
    base_model.trainable = False

    # Create the complete model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc')
        ]
    )

    return model

# Create the model
xception_model = create_xception_model()

# Display model summary
print("\nXception Model Summary:")
xception_model.summary()

## Define Callbacks for Training

In [ ]:
def setup_callbacks():
    """
    Set up callbacks for efficient training
    """
    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    )

    # Reduce learning rate when validation loss plateaus
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )

    # Save the best model
    checkpoint = ModelCheckpoint(
        'best_xception_pneumonia_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    return [early_stop, reduce_lr, checkpoint]

callbacks = setup_callbacks()
print("Callbacks setup complete!")

## Train the Model

In [ ]:
print("Starting Xception model training...")
EPOCHS = 30

# Calculate steps per epoch
train_steps = train_generator.samples // BATCH_SIZE
val_steps = validation_generator.samples // BATCH_SIZE

print(f"Training steps per epoch: {train_steps}")
print(f"Validation steps per epoch: {val_steps}")
print(f"Maximum epochs: {EPOCHS}")

# Start timer
start_time = time.time()

# Train the model
history = xception_model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=val_steps,
    callbacks=callbacks,
    class_weight=class_weights,  # Important for handling imbalance
    verbose=1
)

training_time = time.time() - start_time
print(f"\nXception training completed in {training_time/60:.2f} minutes!")

## Evaluate the Model

In [ ]:
def evaluate_xception_model_safe(model, test_generator):
    """
    Ultra-safe evaluation that handles any metric naming issues
    """
    print("\nEvaluating Xception model on test set...")

    # Get the actual metric names
    print("Available metrics:", model.metrics_names)

    # Evaluate and get results as dictionary (TensorFlow 2.4+)
    try:
        test_results = model.evaluate(test_generator, verbose=1, return_dict=True)
        print("Results as dictionary:", test_results)
    except:
        # Fallback for older TensorFlow versions
        test_results = model.evaluate(test_generator, verbose=1)
        metrics_dict = dict(zip(model.metrics_names, test_results))
        print("Results as list:", test_results)
        print("Metrics dictionary:", metrics_dict)
        test_results = metrics_dict

    print(f"\nXception Test Results:")

    # Extract all available metrics
    for metric_name, value in test_results.items():
        print(f"{metric_name}: {value:.4f}")

    # Calculate F1-score from available metrics
    precision = test_results.get('precision', 0.0)
    recall = test_results.get('recall', 0.0)

    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
        print(f"F1-Score: {f1_score:.4f}")

    # Make predictions
    print("\nMaking predictions...")
    test_generator.reset()
    predictions = model.predict(test_generator, verbose=1)
    predicted_classes = (predictions > 0.5).astype(int).flatten()

    # True labels
    true_classes = test_generator.classes

    # Classification report
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes,
                              target_names=['NORMAL', 'PNEUMONIA'], digits=4))

    return test_results, predictions, predicted_classes

# Use the safe evaluation function
test_metrics, predictions, predicted_classes = evaluate_xception_model_safe(xception_model, test_generator)

## Visualize Results

In [ ]:
def plot_training_history_simple(history):
    """
    Plot comprehensive training history without fine-tuning
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, color='blue')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='red')
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss
    axes[0, 1].plot(history.history['loss'], label='Training Loss', linewidth=2, color='blue')
    axes[0, 1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='red')
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision
    if 'precision' in history.history:
        axes[0, 2].plot(history.history['precision'], label='Training Precision', linewidth=2, color='blue')
        axes[0, 2].plot(history.history['val_precision'], label='Validation Precision', linewidth=2, color='red')
        axes[0, 2].set_title('Model Precision', fontsize=14, fontweight='bold')
        axes[0, 2].set_xlabel('Epoch')
        axes[0, 2].set_ylabel('Precision')
        axes[0, 2].legend()
        axes[0, 2].grid(True, alpha=0.3)
    else:
        axes[0, 2].text(0.5, 0.5, 'Precision data\nnot available',
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[0, 2].transAxes, fontsize=12)
        axes[0, 2].set_title('Model Precision', fontsize=14, fontweight='bold')

    # Recall
    if 'recall' in history.history:
        axes[1, 0].plot(history.history['recall'], label='Training Recall', linewidth=2, color='blue')
        axes[1, 0].plot(history.history['val_recall'], label='Validation Recall', linewidth=2, color='red')
        axes[1, 0].set_title('Model Recall', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Recall')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'Recall data\nnot available',
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[1, 0].transAxes, fontsize=12)
        axes[1, 0].set_title('Model Recall', fontsize=14, fontweight='bold')

    # AUC
    if 'auc' in history.history:
        axes[1, 1].plot(history.history['auc'], label='Training AUC', linewidth=2, color='blue')
        axes[1, 1].plot(history.history['val_auc'], label='Validation AUC', linewidth=2, color='red')
        axes[1, 1].set_title('Model AUC', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('AUC')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'AUC data\nnot available',
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[1, 1].transAxes, fontsize=12)
        axes[1, 1].set_title('Model AUC', fontsize=14, fontweight='bold')

    # Learning Rate
    if 'lr' in history.history:
        axes[1, 2].plot(history.history['lr'], label='Learning Rate', linewidth=2, color='purple')
        axes[1, 2].set_title('Learning Rate', fontsize=14, fontweight='bold')
        axes[1, 2].set_xlabel('Epoch')
        axes[1, 2].set_ylabel('Learning Rate')
        axes[1, 2].set_yscale('log')
        axes[1, 2].legend()
        axes[1, 2].grid(True, alpha=0.3)
    else:
        # Show training summary instead
        final_train_acc = history.history['accuracy'][-1]
        final_val_acc = history.history['val_accuracy'][-1]
        final_train_loss = history.history['loss'][-1]
        final_val_loss = history.history['val_loss'][-1]

        summary_text = f"Final Training Accuracy: {final_train_acc:.4f}\nFinal Validation Accuracy: {final_val_acc:.4f}\nFinal Training Loss: {final_train_loss:.4f}\nFinal Validation Loss: {final_val_loss:.4f}"

        axes[1, 2].text(0.5, 0.5, summary_text,
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[1, 2].transAxes, fontsize=11,
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
        axes[1, 2].set_title('Training Summary', fontsize=14, fontweight='bold')
        axes[1, 2].set_xticks([])
        axes[1, 2].set_yticks([])

    plt.suptitle('Xception Training History - Pneumonia Detection', fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history_simple(history)

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix_detailed(test_generator, predicted_classes):
    """
    Plot detailed confusion matrix with percentages
    """
    cm = confusion_matrix(test_generator.classes, predicted_classes)

    # Calculate percentages
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Plot absolute values
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=['NORMAL', 'PNEUMONIA'],
                yticklabels=['NORMAL', 'PNEUMONIA'],
                annot_kws={"size": 14})
    ax1.set_title('Confusion Matrix (Absolute Values)\nXception Model',
                  fontsize=14, fontweight='bold', pad=20)
    ax1.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
    ax1.set_ylabel('True Label', fontsize=12, fontweight='bold')

    # Plot percentages
    sns.heatmap(cm_percentage, annot=True, fmt='.2f', cmap='Greens', ax=ax2,
                xticklabels=['NORMAL', 'PNEUMONIA'],
                yticklabels=['NORMAL', 'PNEUMONIA'],
                annot_kws={"size": 12})
    ax2.set_title('Confusion Matrix (Percentages)\nXception Model',
                  fontsize=14, fontweight='bold', pad=20)
    ax2.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
    ax2.set_ylabel('True Label', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Print numerical analysis
    print("Confusion Matrix Analysis:")
    print(f"True Negatives (Normal correctly classified): {cm[0,0]} ({cm_percentage[0,0]:.1f}%)")
    print(f"False Positives (Normal misclassified as Pneumonia): {cm[0,1]} ({cm_percentage[0,1]:.1f}%)")
    print(f"False Negatives (Pneumonia misclassified as Normal): {cm[1,0]} ({cm_percentage[1,0]:.1f}%)")
    print(f"True Positives (Pneumonia correctly classified): {cm[1,1]} ({cm_percentage[1,1]:.1f}%)")

plot_confusion_matrix_detailed(test_generator, predicted_classes)

In [ ]:
# Plot ROC Curve
def plot_roc_curve(test_generator, predictions):
    """
    Plot ROC curve for model performance
    """
    fpr, tpr, thresholds = roc_curve(test_generator.classes, predictions)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Xception - Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.show()

    return roc_auc

roc_auc = plot_roc_curve(test_generator, predictions)
print(f"ROC AUC Score: {roc_auc:.4f}")

## Sample Predictions Visualization

In [ ]:
def display_sample_predictions(model, test_generator, num_samples=8):
    """
    Display sample predictions with true and predicted labels
    """
    # Get a batch of test data
    test_generator.reset()
    x_batch, y_batch = next(test_generator)

    # Make predictions for this batch
    predictions = model.predict(x_batch[:num_samples], verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).flatten()
    true_labels = y_batch[:num_samples].astype(int)

    # Class names
    class_names_dict = {0: 'NORMAL', 1: 'PNEUMONIA'}

    # Plot samples
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()

    for i in range(num_samples):
        axes[i].imshow(x_batch[i])
        axes[i].axis('off')

        true_class = class_names_dict[true_labels[i]]
        pred_class = class_names_dict[predicted_labels[i]]
        confidence = predictions[i][0]

        # Color code: green for correct, red for incorrect
        color = 'green' if true_labels[i] == predicted_labels[i] else 'red'

        title = f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.3f}'
        axes[i].set_title(title, color=color, fontsize=11, fontweight='bold', pad=10)

    plt.suptitle('Xception - Sample Predictions on Test Set',
                 fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

display_sample_predictions(xception_model, test_generator)

## Save the Model and Results

In [ ]:
def save_xception_results_simple(model, history, test_metrics):
    """
    Save model, training history, and metrics without fine-tuning
    """
    print("Saving Xception results...")

    # Save the model
    model.save('xception_pneumonia_detection.h5')
    print("Model saved as 'xception_pneumonia_detection.h5'")

    # Save training history to CSV
    history_df = pd.DataFrame(history.history)
    history_df['epoch'] = range(1, len(history_df) + 1)
    history_df.to_csv('xception_training_history.csv', index=False)
    print("Training history saved as 'xception_training_history.csv'")

    # Save test metrics
    metrics_df = pd.DataFrame([test_metrics])
    metrics_df.to_csv('xception_test_metrics.csv', index=False)
    print("Test metrics saved as 'xception_test_metrics.csv'")

    # Create performance summary
    summary = {
        'final_val_accuracy': history.history['val_accuracy'][-1],
        'final_val_loss': history.history['val_loss'][-1],
        'test_accuracy': test_metrics['accuracy'],
        'test_precision': test_metrics['precision'],
        'test_recall': test_metrics['recall'],
        'test_auc': test_metrics['auc'],
        'training_time_minutes': training_time / 60,
        'model_parameters': model.count_params()
    }

    summary_df = pd.DataFrame([summary])
    summary_df.to_csv('xception_performance_summary.csv', index=False)
    print("Performance summary saved as 'xception_performance_summary.csv'")

# Save the results without fine-tuning data
save_xception_results_simple(xception_model, history, test_metrics)

## Performance Summary

In [ ]:
# Step 14: Display final performance summary
def display_xception_summary():
    """
    Display comprehensive project summary for Xception
    """
    print("=" * 70)
    print("XCEPTION PNEUMONIA DETECTION - PROJECT SUMMARY")
    print("=" * 70)

    print(f"\nDATASET STATISTICS:")
    print(f"   Training samples: {train_generator.samples}")
    print(f"   Validation samples: {validation_generator.samples}")
    print(f"   Test samples: {test_generator.samples}")
    print(f"   Classes: {class_names}")
    print(f"   Class weights: {class_weights}")

    print(f"\nMODEL ARCHITECTURE:")
    print(f"   Base Model: Xception (pre-trained on ImageNet)")
    print(f"   Input Size: {IMG_HEIGHT}x{IMG_WIDTH}x3")
    print(f"   Trainable Parameters: {xception_model.trainable_weights.__len__()} layers")
    print(f"   Total Parameters: {xception_model.count_params():,}")
    print(f"   Key Feature: Depthwise Separable Convolutions")

    print(f"\nTRAINING PERFORMANCE:")
    print(f"   Training Time: {training_time/60:.2f} minutes")
    print(f"   Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"   Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

    print(f"\nTEST PERFORMANCE:")
    print(f"   Test Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"   Test Precision: {test_metrics['precision']:.4f}")
    print(f"   Test Recall: {test_metrics['recall']:.4f}")
    print(f"   Test AUC: {test_metrics['auc']:.4f}")
    print(f"   ROC AUC: {roc_auc:.4f}")

    f1 = 2 * (test_metrics['precision'] * test_metrics['recall']) / (test_metrics['precision'] + test_metrics['recall'])
    print(f"   Test F1-Score: {f1:.4f}")

    print(f"\nSAVED FILES:")
    print("   1. xception_pneumonia_detection.h5 - Trained model")
    print("   2. xception_training_history.csv - Training metrics history")
    print("   3. xception_test_metrics.csv - Detailed test metrics")
    print("   4. xception_performance_summary.csv - Performance summary")

display_xception_summary()

# Fine-tuning section

## Fine-tuning start

In [ ]:
# Fine-tuning section
def fine_tune_model(model_to_tune, base_layers_unfreeze=30):
    """
    Unfreeze the top layers of the base model for fine-tuning
    """
    print("Inside fine_tune_model function...")
    if model_to_tune is None:
        print("Error: Model passed to fine_tune_model is None.")
        return None

    # Unfreeze the top N layers of the base model
    model_to_tune.trainable = True
    print(f"Initial trainable status of model_to_tune: {model_to_tune.trainable}")

    # Freeze the bottom layers, unfreeze the top layers
    # Check if the model has a base model as the first layer
    if isinstance(model_to_tune.layers[0], tf.keras.Model):
        print("First layer is a Keras Model (assuming base model).")
        base_model_layers = model_to_tune.layers[0].layers
        print(f"Total layers in base model: {len(base_model_layers)}")
        # Freeze all layers in the base model first
        for layer in base_model_layers:
            layer.trainable = False
        print("All base model layers initially frozen.")
        # Unfreeze the top layers of the base model
        unfrozen_count = 0
        for layer in base_model_layers[-base_layers_unfreeze:]:
            layer.trainable = True
            unfrozen_count += 1
        print(f"Unfrozen top {unfrozen_count} layers of the base model for fine-tuning")
    else:
        print("First layer is not a Keras Model. Unfreezing layers in the sequential model directly.")
        # If no explicit base model layer, unfreeze the last 'base_layers_unfreeze' layers of the sequential model
        total_layers = len(model_to_tune.layers)
        layers_to_freeze = max(0, total_layers - base_layers_unfreeze)
        for layer in model_to_tune.layers[:layers_to_freeze]:
            layer.trainable = False
        for layer in model_to_tune.layers[layers_to_freeze:]:
             layer.trainable = True
        print(f"Unfrozen top {total_layers - layers_to_freeze} layers of the sequential model for fine-tuning")


    # Recompile with a lower learning rate
    print("Recompiling model with lower learning rate...")
    model_to_tune.compile(
        optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    print(f"Trainable layers after unfreezing and recompile: {sum([layer.trainable for layer in model_to_tune.layers])}")
    print(f"Total layers: {len(model_to_tune.layers)}")
    print("fine_tune_model function finished.")

    return model_to_tune

# Fine-tune the model
print("Starting fine-tuning...")
# Pass the created xception_model to the fine-tuning function
# Ensure xception_model is defined and not None from previous steps
if 'xception_model' not in locals() or xception_model is None:
    print("Error: xception_model is not defined or is None. Please run the model building cell first.")
    fine_tuned_model = None
else:
    fine_tuned_model = fine_tune_model(xception_model, base_layers_unfreeze=50)

# Proceed with training only if fine_tuned_model was successfully created
if fine_tuned_model is not None:
    # Callbacks for fine-tuning
    fine_tune_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            '/content/drive/MyDrive/DL Assignment/xception_fine_tuned_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
    ]

    # Fine-tuning training
    print("Starting fine-tuning training...")
    fine_tune_history = fine_tuned_model.fit(
        train_generator, # Use train_generator
        steps_per_epoch=len(train_generator),
        epochs=50,
        validation_data=validation_generator,  # Use validation_generator here
        validation_steps=len(validation_generator), # Use validation_generator here
        callbacks=fine_tune_callbacks,
        verbose=1
    )

    # Save the fine-tuned model
    fine_tuned_model.save('/content/drive/MyDrive/DL Assignment/xception_fine_tuned_final.h5')
    print("Fine-tuned model saved!")
else:
    print("Fine-tuning model creation failed. Skipping training.")

In [ ]:
# Plot fine-tuning training history
def plot_fine_tune_training(history):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot training & validation accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0].set_title('Fine-tuning Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    # Plot training & validation loss
    axes[1].plot(history.history['loss'], label='Training Loss')
    axes[1].plot(history.history['val_loss'], label='Validation Loss')
    axes[1].set_title('Fine-tuning Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

plot_fine_tune_training(fine_tune_history)

## comparison between initial training and fine-tuning

In [ ]:
# Compare initial training vs fine-tuning
def compare_training_histories(initial_history, fine_tune_history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy comparison
    axes[0, 0].plot(initial_history.history['accuracy'], label='Initial Training', alpha=0.7)
    axes[0, 0].plot(initial_history.history['val_accuracy'], label='Initial Validation', alpha=0.7)
    axes[0, 0].set_title('Initial Training - Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(fine_tune_history.history['accuracy'], label='Fine-tuning Training', alpha=0.7)
    axes[0, 1].plot(fine_tune_history.history['val_accuracy'], label='Fine-tuning Validation', alpha=0.7)
    axes[0, 1].set_title('Fine-tuning - Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    # Loss comparison
    axes[1, 0].plot(initial_history.history['loss'], label='Initial Training', alpha=0.7)
    axes[1, 0].plot(initial_history.history['val_loss'], label='Initial Validation', alpha=0.7)
    axes[1, 0].set_title('Initial Training - Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    axes[1, 1].plot(fine_tune_history.history['loss'], label='Fine-tuning Training', alpha=0.7)
    axes[1, 1].plot(fine_tune_history.history['val_loss'], label='Fine-tuning Validation', alpha=0.7)
    axes[1, 1].set_title('Fine-tuning - Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.show()

# Compare if both histories are available
compare_training_histories(history, fine_tune_history)

## evaluation of fine-tuned model

In [ ]:
# Evaluate fine-tuned model
print("Evaluating fine-tuned model...")
fine_tune_test_loss, fine_tune_test_accuracy, fine_tune_test_precision, fine_tune_test_recall = fine_tuned_model.evaluate(
    test_generator,
    steps=len(test_generator),
    verbose=1
)

fine_tune_test_f1 = 2 * (fine_tune_test_precision * fine_tune_test_recall) / (fine_tune_test_precision + fine_tune_test_recall)

print(f"\nFine-tuned Model Test Results:")
print(f"Accuracy: {fine_tune_test_accuracy:.4f}")
print(f"Precision: {fine_tune_test_precision:.4f}")
print(f"Recall: {fine_tune_test_recall:.4f}")
print(f"F1-Score: {fine_tune_test_f1:.4f}")

# Compare with initial model results
print(f"\nComparison with Initial Model:")
print(f"Accuracy:  {test_accuracy:.4f} (initial) -> {fine_tune_test_accuracy:.4f} (fine-tuned)")
print(f"Precision: {test_precision:.4f} (initial) -> {fine_tune_test_precision:.4f} (fine-tuned)")
print(f"Recall:    {test_recall:.4f} (initial) -> {fine_tune_test_recall:.4f} (fine-tuned)")
print(f"F1-Score:  {test_f1:.4f} (initial) -> {fine_tune_test_f1:.4f} (fine-tuned)")

## fine-tuning analysis

In [ ]:
# Detailed analysis of fine-tuning impact
def analyze_fine_tuning_impact():
    # Get predictions from both models for comparison
    test_generator.reset()
    # Use xception_model for initial predictions
    initial_preds = xception_model.predict(test_generator, steps=len(test_generator), verbose=1)
    test_generator.reset()
    fine_tune_preds = fine_tuned_model.predict(test_generator, steps=len(test_generator), verbose=1)

    # Convert predictions to binary
    initial_binary_preds = (initial_preds > 0.5).astype(int)
    fine_tune_binary_preds = (fine_tune_preds > 0.5).astype(int)

    # Get true labels
    true_labels = test_generator.classes[:len(initial_binary_preds)]

    # Calculate confusion matrices
    initial_cm = confusion_matrix(true_labels, initial_binary_preds)
    fine_tune_cm = confusion_matrix(true_labels, fine_tune_binary_preds)

    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Initial model confusion matrix
    sns.heatmap(initial_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title('Initial Model - Confusion Matrix')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')

    # Fine-tuned model confusion matrix
    sns.heatmap(fine_tune_cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
    axes[1].set_title('Fine-tuned Model - Confusion Matrix')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')

    plt.tight_layout()
    plt.show()

    # Print improvement analysis
    initial_tn, initial_fp, initial_fn, initial_tp = initial_cm.ravel()
    fine_tune_tn, fine_tune_fp, fine_tune_fn, fine_tune_tp = fine_tune_cm.ravel()

    print("\nImprovement Analysis:")
    print(f"True Positives:  {initial_tp} -> {fine_tune_tp} (Change: {fine_tune_tp - initial_tp:+d})")
    print(f"False Negatives: {initial_fn} -> {fine_tune_fn} (Change: {fine_tune_fn - initial_fn:+d})")
    print(f"False Positives: {initial_fp} -> {fine_tune_fp} (Change: {fine_tune_fp - initial_fp:+d})")
    print(f"True Negatives:  {initial_tn} -> {fine_tune_tn} (Change: {fine_tune_tn - initial_tn:+d})")

analyze_fine_tuning_impact()